## dataload

In [72]:
import mlflow
import mlflow.lightgbm
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import os
import random
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from utils import custom_train_test_split, lgbm_predict, post_slack, title2filename

SEED=13
## 1. 데이터 로딩
data_dir = '/opt/ml/input/data' # 경로
after_fe_path = os.path.join(data_dir, 'after_fe_train_test_bigcategory_fe.pkl')
df = pd.read_pickle(after_fe_path)

train_df = df[df.kind=='train']
train, valid = custom_train_test_split(train_df, ratio=0.7, seed=SEED) # 훈련데이터 split
test = df[df.kind=='test'] # 테스트 데이터
train2 = test[test.answerCode!=-1] # 테스트데이터 마지막 제출 2번쨰꺼까지 훈련데이터로 사용
train = pd.concat([train,train2]) # 훈련데이터 병합
train.shape, valid.shape, test.shape

x_train = train.drop('answerCode',axis=1)
y_train = train[['answerCode']]

x_valid = valid.drop('answerCode',axis=1)
y_valid = valid[['answerCode']]
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

((1845539, 40), (1974, 40), (260114, 40))

((1845539, 39), (1845539, 1), (1974, 39), (1974, 1))

In [2]:
df[:3]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,kind,uidIdx,assIdx,testIdx,...,Timestamp2,solvetime,solvesec,solvesec_3600,time_category,solvesec_cumsum,solvecumsum_category,big_category_acc,big_category_std,big_category_cumconut
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,train,0,5354,975,...,2020-03-24 00:17:11,0 days 00:00:00,0.0,0.0,0,0.0,0,0.711898,0.453371,0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,train,0,5355,975,...,2020-03-24 00:17:14,0 days 00:00:03,3.0,3.0,1,3.0,1,0.711898,0.453371,1
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,train,0,5356,975,...,2020-03-24 00:17:22,0 days 00:00:08,8.0,8.0,3,11.0,4,0.711898,0.453371,2


## Hyper Parameter 설정

In [79]:
# Hyper parameter 설정
params = {
#     "max_depth": 8,  # 8,
#     "min_data_in_leaf": 1000,
    # "feature_fraction": 0.6,  # 0.8,
#     "bagging_fraction": 0.75,
    # "max_cat_group": 64,
    "objective": "binary",
#     "boosting": "gbdt",  # dart
#     "learning_rate": 0.01,  # 0.01,
    # "bagging_freq": 5,
    "seed": 42,
    # "max_bin": 50,
#     "num_leaves": 80,  # 40,
#     "metric": "auc",
}

##  big_category 정답률, std, cumcount 추가

In [4]:
x_train.columns

Index(['userID', 'assessmentItemID', 'testId', 'Timestamp', 'KnowledgeTag',
       'kind', 'uidIdx', 'assIdx', 'testIdx', 'user_correct_answer',
       'user_total_answer', 'user_acc', 'month', 'day', 'hour', 'dayname',
       'big_category', 'problem_num', 'mid_category', 'test_mean', 'test_std',
       'test_sum', 'tag_mean', 'tag_std', 'tag_sum', 'Timestamp2', 'solvetime',
       'solvesec', 'solvesec_3600', 'time_category', 'solvesec_cumsum',
       'solvecumsum_category', 'big_category_acc', 'big_category_std',
       'big_category_cumconut'],
      dtype='object')

In [5]:
### 피처 설정
# 사용할 Feature 설정
FEATS = ['uidIdx',
         'assIdx',
         'testIdx',
         'KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
         'solvesec_cumsum',
         'solvecumsum_category',
         'big_category_acc',
         'big_category_std',
         'big_category_cumconut'
        ]

cat_feats = ['uidIdx','assIdx','testIdx','KnowledgeTag','big_category','mid_category',
             'problem_num','dayname','month','time_category','solvecumsum_category']
cat_feats_idx = [i for i,e in enumerate(FEATS) if e in cat_feats]

### 학습 및 예측

date = datetime.now().strftime('%m/%d %a')
title=f"🌈({date})[LGBM big_category 정답률, std, cumcount 추가] 피처: {len(FEATS)}개"
using_feats=", ".join(FEATS)
desc=f"사용된 피처({len(FEATS)})\n{using_feats}"
cat_feats_idx, title, desc

mlflow.lightgbm.autolog()
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

with mlflow.start_run(run_name=title, description=desc) as run:
    model1 = lgb.train(
        params, 
        lgb_x_train,
        valid_sets=[lgb_x_valid],
        verbose_eval=100,
        num_boost_round=3200,
        categorical_feature=cat_feats_idx
    #     early_stopping_rounds=200,
    )

    preds = model1.predict(x_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)
    print(f'VALID AUC : {auc} ACC : {acc}\n')
    mlflow.log_metric("VAL AUC",auc)
    mlflow.log_metric("VAL Set SEED",SEED)
    model1_run_id=run.info.run_id
    post_slack("done")
    print(f"{model1_run_id=}")
    file_name = title2filename(title)


([0, 1, 2, 3, 6, 7, 8, 9, 11, 21, 23],
 '🌈(11/30 Wed)[LGBM big_category 정답률, std, cumcount 추가] 피처: 27개',
 '사용된 피처(27)\nuidIdx, assIdx, testIdx, KnowledgeTag, user_correct_answer, user_total_answer, big_category, mid_category, problem_num, month, day, dayname, hour, user_acc, test_mean, test_sum, test_std, tag_std, tag_mean, tag_sum, solvesec_3600, time_category, solvesec_cumsum, solvecumsum_category, big_category_acc, big_category_std, big_category_cumconut')

[LightGBM] [Info] Number of positive: 1208276, number of negative: 637263
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030320 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19316
[LightGBM] [Info] Number of data points in the train set: 1845539, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654701 -> initscore=0.639767
[LightGBM] [Info] Start training from score 0.639767
[100]	valid_0's binary_logloss: 0.560243
[200]	valid_0's binary_logloss: 0.551904
[300]	valid_0's binary_logloss: 0.549396
[400]	valid_0's binary_logloss: 0.547937
[500]	valid_0's binary_logloss: 0.546311
[600]	valid_0's binary_logloss: 0.544856
[700]	valid_0's binary_logloss: 0.544792
[800]	valid_0's binary_logloss: 0.544924
[900]	valid_0's binary_logloss: 0.545558
[1000]	valid_0's binary_logloss: 0.545992
[1100]	valid_0's binary_

NameError: name 'model4' is not defined

In [6]:
lgbm_predict(test, model1, FEATS, f'{file_name}.csv')

writing prediction : output/11_30_Wed_LGBM_big_category_정답률_std_cumcount_추가_피처_27개.csv


In [13]:
### 제출 mlflow 등록
# 제출시
LB_AUC=0.8111
run_id = model1_run_id
mlflow.start_run(run_id=run_id)
run = mlflow.active_run()
print("Active run_id: {}".format(run.info.run_id))
mlflow.log_metric("LB AUC",LB_AUC)
mlflow.end_run()

<ActiveRun: >

Active run_id: 6a42312b2c7044a8b69dc362a392f4c8


## big_category 정답률, std, cumcount 추가 - earlystop

In [75]:
x_train[['big_category_acc','big_category_std','big_category_cumconut']].describe().T

,count,mean,std,min,25%,50%,75%,max
big_category_acc,1845539.0,0.654365,0.105236,0.454470,0.521167,0.679714,0.711898,0.801241
big_category_std,1845539.0,0.463369,0.031221,0.399685,0.453371,0.467279,0.498214,0.500802
big_category_cumconut,1845539.0,159.338010,159.836556,0.000000,41.000000,110.000000,228.000000,1268.000000


In [73]:
### 피처 설정
# 사용할 Feature 설정
FEATS = ['uidIdx',
         'assIdx',
         'testIdx',
         'KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
         'solvesec_cumsum',
         'solvecumsum_category',
         'big_category_acc',
         'big_category_std',
         'big_category_cumconut'
        ]

cat_feats = ['uidIdx','assIdx','testIdx','KnowledgeTag','big_category','mid_category',
             'problem_num','dayname','month','time_category','solvecumsum_category']
cat_feats_idx = [i for i,e in enumerate(FEATS) if e in cat_feats]

### 학습 및 예측

date = datetime.now().strftime('%m/%d %a')
title=f"🌈({date})[LGBM big_category 정답률, std, cumcount 추가 earlystop] 피처: {len(FEATS)}개"
using_feats=", ".join(FEATS)
desc=f"사용된 피처({len(FEATS)})\n{using_feats}"
cat_feats_idx, title, desc

mlflow.lightgbm.autolog()
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

with mlflow.start_run(run_name=title, description=desc) as run:
    model2re = lgb.train(
        params, 
        lgb_x_train,
        valid_sets=[lgb_x_valid],
        verbose_eval=100,
        num_boost_round=3200,
        categorical_feature=cat_feats_idx,
        early_stopping_rounds=200,
    )

    preds = model2re.predict(x_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)
    print(f'VALID AUC : {auc} ACC : {acc}\n')
    mlflow.log_metric("VAL AUC",auc)
    mlflow.log_metric("VAL Set SEED",SEED)
    model2re_run_id=run.info.run_id
    post_slack("done")
    print(f"{model2re_run_id=}")
    file_name = title2filename(title)
    lgbm_predict(test, model2re, FEATS, f'{file_name}.csv')

([0, 1, 2, 3, 6, 7, 8, 9, 11, 21, 23],
 '🌈(12/02 Fri)[LGBM big_category 정답률, std, cumcount 추가 earlystop] 피처: 27개',
 '사용된 피처(27)\nuidIdx, assIdx, testIdx, KnowledgeTag, user_correct_answer, user_total_answer, big_category, mid_category, problem_num, month, day, dayname, hour, user_acc, test_mean, test_sum, test_std, tag_std, tag_mean, tag_sum, solvesec_3600, time_category, solvesec_cumsum, solvecumsum_category, big_category_acc, big_category_std, big_category_cumconut')

[LightGBM] [Info] Number of positive: 1208276, number of negative: 637263
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030669 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19316
[LightGBM] [Info] Number of data points in the train set: 1845539, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654701 -> initscore=0.639767
[LightGBM] [Info] Start training from score 0.639767
Training until validation scores don't improve for 200 rounds
[100]	valid_0's binary_logloss: 0.560243
[200]	valid_0's binary_logloss: 0.551904
[300]	valid_0's binary_logloss: 0.549396
[400]	valid_0's binary_logloss: 0.547937
[500]	valid_0's binary_logloss: 0.546311
[600]	valid_0's binary_logloss: 0.544856
[700]	valid_0's binary_logloss: 0.544792
[800]	valid_0's binary_logloss: 0.544924
Early stopping, best iteration is:
[672]	vali

In [8]:
### 피처 설정
# 사용할 Feature 설정
FEATS = ['uidIdx',
         'assIdx',
         'testIdx',
         'KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
         'solvesec_cumsum',
         'solvecumsum_category',
         'big_category_acc',
         'big_category_std',
         'big_category_cumconut'
        ]

cat_feats = ['uidIdx','assIdx','testIdx','KnowledgeTag','big_category','mid_category',
             'problem_num','dayname','month','time_category','solvecumsum_category']
cat_feats_idx = [i for i,e in enumerate(FEATS) if e in cat_feats]

### 학습 및 예측

date = datetime.now().strftime('%m/%d %a')
title=f"🌈({date})[LGBM big_category 정답률, std, cumcount 추가 earlystop] 피처: {len(FEATS)}개"
using_feats=", ".join(FEATS)
desc=f"사용된 피처({len(FEATS)})\n{using_feats}"
cat_feats_idx, title, desc

mlflow.lightgbm.autolog()
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

with mlflow.start_run(run_name=title, description=desc) as run:
    model2 = lgb.train(
        params, 
        lgb_x_train,
        valid_sets=[lgb_x_valid],
        verbose_eval=100,
        num_boost_round=3200,
        categorical_feature=cat_feats_idx,
        early_stopping_rounds=200,
    )

    preds = model2.predict(x_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)
    print(f'VALID AUC : {auc} ACC : {acc}\n')
    mlflow.log_metric("VAL AUC",auc)
    mlflow.log_metric("VAL Set SEED",SEED)
    model2_run_id=run.info.run_id
    post_slack("done")
    print(f"{model2_run_id=}")
    file_name = title2filename(title)
    lgbm_predict(test, model2, FEATS, f'{file_name}.csv')

([0, 1, 2, 3, 6, 7, 8, 9, 11, 21, 23],
 '🌈(11/30 Wed)[LGBM big_category 정답률, std, cumcount 추가 earlystop] 피처: 27개',
 '사용된 피처(27)\nuidIdx, assIdx, testIdx, KnowledgeTag, user_correct_answer, user_total_answer, big_category, mid_category, problem_num, month, day, dayname, hour, user_acc, test_mean, test_sum, test_std, tag_std, tag_mean, tag_sum, solvesec_3600, time_category, solvesec_cumsum, solvecumsum_category, big_category_acc, big_category_std, big_category_cumconut')

[LightGBM] [Info] Number of positive: 1208276, number of negative: 637263
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030329 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19316
[LightGBM] [Info] Number of data points in the train set: 1845539, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654701 -> initscore=0.639767
[LightGBM] [Info] Start training from score 0.639767
Training until validation scores don't improve for 200 rounds
[100]	valid_0's binary_logloss: 0.560243
[200]	valid_0's binary_logloss: 0.551904
[300]	valid_0's binary_logloss: 0.549396
[400]	valid_0's binary_logloss: 0.547937
[500]	valid_0's binary_logloss: 0.546311
[600]	valid_0's binary_logloss: 0.544856
[700]	valid_0's binary_logloss: 0.544792
[800]	valid_0's binary_logloss: 0.544924
Early stopping, best iteration is:
[672]	vali

In [12]:
### 제출 mlflow 등록
# 제출시
LB_AUC=0.8155
run_id = model2_run_id
mlflow.start_run(run_id=run_id)
run = mlflow.active_run()
print("Active run_id: {}".format(run.info.run_id))
mlflow.log_metric("LB AUC",LB_AUC)
mlflow.end_run()

<ActiveRun: >

Active run_id: 88aac4c9c9cb42c089856d46ef8ba746


## 기존 month 추가한것 까지만 earlystop

In [71]:
test.shape

(260114, 40)

In [10]:
### 피처 설정
# 사용할 Feature 설정
FEATS = ['uidIdx',
         'assIdx',
         'testIdx',
         'KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
         'solvesec_cumsum',
         'solvecumsum_category',
        ]

cat_feats = ['uidIdx','assIdx','testIdx','KnowledgeTag','big_category','mid_category',
             'problem_num','dayname','month','time_category','solvecumsum_category']
cat_feats_idx = [i for i,e in enumerate(FEATS) if e in cat_feats]

### 학습 및 예측

date = datetime.now().strftime('%m/%d %a')
title=f"🌈({date})[LGBM month추가한것 까지만 earlystop] 피처: {len(FEATS)}개"
using_feats=", ".join(FEATS)
desc=f"사용된 피처({len(FEATS)})\n{using_feats}"
cat_feats_idx, title, desc

mlflow.lightgbm.autolog()
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

with mlflow.start_run(run_name=title, description=desc) as run:
    model5 = lgb.train(
        params, 
        lgb_x_train,
        valid_sets=[lgb_x_valid],
        verbose_eval=100,
        num_boost_round=3200,
        categorical_feature=cat_feats_idx,
        early_stopping_rounds=200,
    )

    preds = model5.predict(x_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)
    print(f'VALID AUC : {auc} ACC : {acc}\n')
    mlflow.log_metric("VAL AUC",auc)
    mlflow.log_metric("VAL Set SEED",SEED)
    model5_run_id=run.info.run_id
    post_slack("done")
    print(f"{model5_run_id=}")
    file_name = title2filename(title)
    lgbm_predict(test, model5, FEATS, f'{file_name}.csv')

([0, 1, 2, 3, 6, 7, 8, 9, 11, 21, 23],
 '🌈(12/01 Thu)[LGBM month추가한것 까지만 earlystop] 피처: 24개',
 '사용된 피처(24)\nuidIdx, assIdx, testIdx, KnowledgeTag, user_correct_answer, user_total_answer, big_category, mid_category, problem_num, month, day, dayname, hour, user_acc, test_mean, test_sum, test_std, tag_std, tag_mean, tag_sum, solvesec_3600, time_category, solvesec_cumsum, solvecumsum_category')

[LightGBM] [Info] Number of positive: 1208276, number of negative: 637263
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023671 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19041
[LightGBM] [Info] Number of data points in the train set: 1845539, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654701 -> initscore=0.639767
[LightGBM] [Info] Start training from score 0.639767
Training until validation scores don't improve for 200 rounds
[100]	valid_0's binary_logloss: 0.561618
[200]	valid_0's binary_logloss: 0.554541
[300]	valid_0's binary_logloss: 0.552477
[400]	valid_0's binary_logloss: 0.550644
[500]	valid_0's binary_logloss: 0.549608
[600]	valid_0's binary_logloss: 0.549414
[700]	valid_0's binary_logloss: 0.549202
Early stopping, best iteration is:
[543]	valid_0's binary_logloss: 0.548983
VALID AUC 

In [10]:
### 피처 설정
# 사용할 Feature 설정
FEATS = ['uidIdx',
         'assIdx',
         'testIdx',
         'KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
         'solvesec_cumsum',
         'solvecumsum_category',
        ]

cat_feats = ['uidIdx','assIdx','testIdx','KnowledgeTag','big_category','mid_category',
             'problem_num','dayname','month','time_category','solvecumsum_category']
cat_feats_idx = [i for i,e in enumerate(FEATS) if e in cat_feats]

### 학습 및 예측

date = datetime.now().strftime('%m/%d %a')
title=f"🌈({date})[LGBM month추가한것 까지만 earlystop] 피처: {len(FEATS)}개"
using_feats=", ".join(FEATS)
desc=f"사용된 피처({len(FEATS)})\n{using_feats}"
cat_feats_idx, title, desc

mlflow.lightgbm.autolog()
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

with mlflow.start_run(run_name=title, description=desc) as run:
    model5 = lgb.train(
        params, 
        lgb_x_train,
        valid_sets=[lgb_x_valid],
        verbose_eval=100,
        num_boost_round=3200,
        categorical_feature=cat_feats_idx,
        early_stopping_rounds=200,
    )

    preds = model5.predict(x_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)
    print(f'VALID AUC : {auc} ACC : {acc}\n')
    mlflow.log_metric("VAL AUC",auc)
    mlflow.log_metric("VAL Set SEED",SEED)
    model5_run_id=run.info.run_id
    post_slack("done")
    print(f"{model5_run_id=}")
    file_name = title2filename(title)
    lgbm_predict(test, model5, FEATS, f'{file_name}.csv')

([0, 1, 2, 3, 6, 7, 8, 9, 11, 21, 23],
 '🌈(12/01 Thu)[LGBM month추가한것 까지만 earlystop] 피처: 24개',
 '사용된 피처(24)\nuidIdx, assIdx, testIdx, KnowledgeTag, user_correct_answer, user_total_answer, big_category, mid_category, problem_num, month, day, dayname, hour, user_acc, test_mean, test_sum, test_std, tag_std, tag_mean, tag_sum, solvesec_3600, time_category, solvesec_cumsum, solvecumsum_category')

[LightGBM] [Info] Number of positive: 1208276, number of negative: 637263
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023671 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19041
[LightGBM] [Info] Number of data points in the train set: 1845539, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654701 -> initscore=0.639767
[LightGBM] [Info] Start training from score 0.639767
Training until validation scores don't improve for 200 rounds
[100]	valid_0's binary_logloss: 0.561618
[200]	valid_0's binary_logloss: 0.554541
[300]	valid_0's binary_logloss: 0.552477
[400]	valid_0's binary_logloss: 0.550644
[500]	valid_0's binary_logloss: 0.549608
[600]	valid_0's binary_logloss: 0.549414
[700]	valid_0's binary_logloss: 0.549202
Early stopping, best iteration is:
[543]	valid_0's binary_logloss: 0.548983
VALID AUC 

In [11]:
### 제출 mlflow 등록
# 제출시
LB_AUC=0.8112
run_id = model5_run_id
mlflow.start_run(run_id=run_id)
run = mlflow.active_run()
print("Active run_id: {}".format(run.info.run_id))
mlflow.log_metric("LB AUC",LB_AUC)
mlflow.end_run()

<ActiveRun: >

Active run_id: d63b95d1bcd243e9ace084652f35c287


In [18]:
df.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'kind', 'uidIdx', 'assIdx', 'testIdx',
       'user_correct_answer', 'user_total_answer', 'user_acc', 'month', 'day',
       'hour', 'dayname', 'big_category', 'problem_num', 'mid_category',
       'test_mean', 'test_std', 'test_sum', 'tag_mean', 'tag_std', 'tag_sum',
       'Timestamp2', 'solvetime', 'solvesec', 'solvesec_3600', 'time_category',
       'solvesec_cumsum', 'solvecumsum_category', 'big_category_acc',
       'big_category_std', 'big_category_cumconut', 'big_category_user_acc',
       'big_category_user_std', 'big_category_answer',
       'big_category_answer_log1p'],
      dtype='object')

## big category 유저별 정답률, std, cumcount

In [22]:
### 피처 설정
# 사용할 Feature 설정
FEATS = [
         'uidIdx','assIdx','testIdx','KnowledgeTag',
         'user_correct_answer','user_total_answer','user_acc',
         'month','day','dayname','hour',
         'test_mean','test_sum','test_std',
         'tag_std','tag_mean','tag_sum',
         'solvesec_3600','time_category',
         'solvesec_cumsum','solvecumsum_category',
         'big_category',
         'big_category_user_acc','big_category_user_std','big_category_cumconut'
         'mid_category',
         'problem_num',
        ]

cat_feats = ['uidIdx','assIdx','testIdx','KnowledgeTag','big_category','mid_category',
             'problem_num','dayname','month','time_category','solvecumsum_category']
cat_feats_idx = [i for i,e in enumerate(FEATS) if e in cat_feats]

### 학습 및 예측

date = datetime.now().strftime('%m/%d %a')
title=f"🌈({date})[LGBM big category 유저별 정답률, std, cumcount] 피처: {len(FEATS)}개"
using_feats=", ".join(FEATS)
desc=f"사용된 피처({len(FEATS)})\n{using_feats}"
cat_feats_idx, title, desc

mlflow.lightgbm.autolog()
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

with mlflow.start_run(run_name=title, description=desc) as run:
    model6 = lgb.train(
        params, 
        lgb_x_train,
        valid_sets=[lgb_x_valid],
        verbose_eval=100,
        num_boost_round=3200,
        categorical_feature=cat_feats_idx,
        early_stopping_rounds=200,
    )

    preds = model6.predict(x_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)
    print(f'VALID AUC : {auc} ACC : {acc}\n')
    mlflow.log_metric("VAL AUC",auc)
    mlflow.log_metric("VAL Set SEED",SEED)
    model6_run_id=run.info.run_id
    post_slack("done")
    print(f"{model6_run_id=}")
    file_name = title2filename(title)
    lgbm_predict(test, model6, FEATS, f'{file_name}.csv')

([0, 1, 2, 3, 6, 7, 8, 9, 11, 21, 23],
 '🌈(12/01 Thu)[LGBM big category 유저별 정답률, std, cumcount] 피처: 27개',
 '사용된 피처(27)\nuidIdx, assIdx, testIdx, KnowledgeTag, user_correct_answer, user_total_answer, big_category, mid_category, problem_num, month, day, dayname, hour, user_acc, test_mean, test_sum, test_std, tag_std, tag_mean, tag_sum, solvesec_3600, time_category, solvesec_cumsum, solvecumsum_category, big_category_user_acc, big_category_user_std, big_category_cumconut')

[LightGBM] [Info] Number of positive: 1208276, number of negative: 637263
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030201 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19806
[LightGBM] [Info] Number of data points in the train set: 1845539, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654701 -> initscore=0.639767
[LightGBM] [Info] Start training from score 0.639767
Training until validation scores don't improve for 200 rounds
[100]	valid_0's binary_logloss: 0.513417
[200]	valid_0's binary_logloss: 0.502819
[300]	valid_0's binary_logloss: 0.500721
[400]	valid_0's binary_logloss: 0.500024
[500]	valid_0's binary_logloss: 0.499942
[600]	valid_0's binary_logloss: 0.500928
Early stopping, best iteration is:
[425]	valid_0's binary_logloss: 0.499722
VALID AUC : 0.8335442074425188 ACC : 0.756332320162

In [25]:
### 제출 mlflow 등록
# 제출시
LB_AUC=0.8024
run_id = model6_run_id
mlflow.start_run(run_id=run_id)
run = mlflow.active_run()
print("Active run_id: {}".format(run.info.run_id))
mlflow.log_metric("LB AUC",LB_AUC)
mlflow.end_run()

<ActiveRun: >

Active run_id: 3af96264ab7544119d567b2ad3ec4921


## big_category_answer_log1p 추가

In [23]:
### 피처 설정
# 사용할 Feature 설정
FEATS = ['uidIdx',
         'assIdx',
         'testIdx',
         'KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
         'solvesec_cumsum',
         'solvecumsum_category',
         'big_category_user_acc',
         'big_category_user_std',
         'big_category_cumconut',
         'big_category_answer_log1p'
        ]

cat_feats = ['uidIdx','assIdx','testIdx','KnowledgeTag','big_category','mid_category',
             'problem_num','dayname','month','time_category','solvecumsum_category']
cat_feats_idx = [i for i,e in enumerate(FEATS) if e in cat_feats]

### 학습 및 예측

date = datetime.now().strftime('%m/%d %a')
title=f"🌈({date})[LGBM big_category_answer_log1p 추가] 피처: {len(FEATS)}개"
using_feats=", ".join(FEATS)
desc=f"사용된 피처({len(FEATS)})\n{using_feats}"
cat_feats_idx, title, desc

mlflow.lightgbm.autolog()
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

with mlflow.start_run(run_name=title, description=desc) as run:
    model7 = lgb.train(
        params, 
        lgb_x_train,
        valid_sets=[lgb_x_valid],
        verbose_eval=100,
        num_boost_round=3200,
        categorical_feature=cat_feats_idx,
        early_stopping_rounds=200,
    )

    preds = model7.predict(x_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)
    print(f'VALID AUC : {auc} ACC : {acc}\n')
    mlflow.log_metric("VAL AUC",auc)
    mlflow.log_metric("VAL Set SEED",SEED)
    model7_run_id=run.info.run_id
    post_slack("done")
    print(f"{model7_run_id=}")
    file_name = title2filename(title)
    lgbm_predict(test, model7, FEATS, f'{file_name}.csv')

([0, 1, 2, 3, 6, 7, 8, 9, 11, 21, 23],
 '🌈(12/01 Thu)[LGBM big_category_answer_log1p 추가] 피처: 28개',
 '사용된 피처(28)\nuidIdx, assIdx, testIdx, KnowledgeTag, user_correct_answer, user_total_answer, big_category, mid_category, problem_num, month, day, dayname, hour, user_acc, test_mean, test_sum, test_std, tag_std, tag_mean, tag_sum, solvesec_3600, time_category, solvesec_cumsum, solvecumsum_category, big_category_user_acc, big_category_user_std, big_category_cumconut, big_category_answer_log1p')

[LightGBM] [Info] Number of positive: 1208276, number of negative: 637263
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030350 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 20061
[LightGBM] [Info] Number of data points in the train set: 1845539, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654701 -> initscore=0.639767
[LightGBM] [Info] Start training from score 0.639767
Training until validation scores don't improve for 200 rounds
[100]	valid_0's binary_logloss: 0.512626
[200]	valid_0's binary_logloss: 0.503799
[300]	valid_0's binary_logloss: 0.500915
[400]	valid_0's binary_logloss: 0.500644
[500]	valid_0's binary_logloss: 0.500709
[600]	valid_0's binary_logloss: 0.500692
[700]	valid_0's binary_logloss: 0.501092
Early stopping, best iteration is:
[545]	valid_0's binary_logloss: 0.500358
VALID AUC 

In [26]:
### 제출 mlflow 등록
# 제출시
LB_AUC=0.8062
run_id = model7_run_id
mlflow.start_run(run_id=run_id)
run = mlflow.active_run()
print("Active run_id: {}".format(run.info.run_id))
mlflow.log_metric("LB AUC",LB_AUC)
mlflow.end_run()

<ActiveRun: >

Active run_id: 9673ba65009845779aed5d57850755e3


### big_category_answer_log1p 추가 3200 epoch

In [28]:
### 피처 설정
# 사용할 Feature 설정
FEATS = ['uidIdx',
         'assIdx',
         'testIdx',
         'KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
         'solvesec_cumsum',
         'solvecumsum_category',
         'big_category_user_acc',
         'big_category_user_std',
         'big_category_cumconut',
         'big_category_answer_log1p'
        ]

cat_feats = ['uidIdx','assIdx','testIdx','KnowledgeTag','big_category','mid_category',
             'problem_num','dayname','month','time_category','solvecumsum_category']
cat_feats_idx = [i for i,e in enumerate(FEATS) if e in cat_feats]

### 학습 및 예측

date = datetime.now().strftime('%m/%d %a')
title=f"🌈({date})[LGBM big_category_answer_log1p 추가 3200epoch] 피처: {len(FEATS)}개"
using_feats=", ".join(FEATS)
desc=f"사용된 피처({len(FEATS)})\n{using_feats}"
cat_feats_idx, title, desc

mlflow.lightgbm.autolog()
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

with mlflow.start_run(run_name=title, description=desc) as run:
    model71 = lgb.train(
        params, 
        lgb_x_train,
        valid_sets=[lgb_x_valid],
        verbose_eval=100,
        num_boost_round=3200,
        categorical_feature=cat_feats_idx,
#         early_stopping_rounds=200,
    )

    preds = model71.predict(x_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)
    print(f'VALID AUC : {auc} ACC : {acc}\n')
    mlflow.log_metric("VAL AUC",auc)
    mlflow.log_metric("VAL Set SEED",SEED)
    model71_run_id=run.info.run_id
    post_slack("done")
    print(f"{model71_run_id=}")
    file_name = title2filename(title)
    lgbm_predict(test, model71, FEATS, f'{file_name}.csv')

([0, 1, 2, 3, 6, 7, 8, 9, 11, 21, 23],
 '🌈(12/01 Thu)[LGBM big_category_answer_log1p 추가 3200epoch] 피처: 28개',
 '사용된 피처(28)\nuidIdx, assIdx, testIdx, KnowledgeTag, user_correct_answer, user_total_answer, big_category, mid_category, problem_num, month, day, dayname, hour, user_acc, test_mean, test_sum, test_std, tag_std, tag_mean, tag_sum, solvesec_3600, time_category, solvesec_cumsum, solvecumsum_category, big_category_user_acc, big_category_user_std, big_category_cumconut, big_category_answer_log1p')

[LightGBM] [Info] Number of positive: 1208276, number of negative: 637263
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031297 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 20061
[LightGBM] [Info] Number of data points in the train set: 1845539, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654701 -> initscore=0.639767
[LightGBM] [Info] Start training from score 0.639767
[100]	valid_0's binary_logloss: 0.512626
[200]	valid_0's binary_logloss: 0.503799
[300]	valid_0's binary_logloss: 0.500915
[400]	valid_0's binary_logloss: 0.500644
[500]	valid_0's binary_logloss: 0.500709
[600]	valid_0's binary_logloss: 0.500692
[700]	valid_0's binary_logloss: 0.501092
[800]	valid_0's binary_logloss: 0.501289
[900]	valid_0's binary_logloss: 0.501496
[1000]	valid_0's binary_logloss: 0.502544
[1100]	valid_0's binary_

In [29]:
### 제출 mlflow 등록
# 제출시
LB_AUC=0.7948
run_id = model71_run_id
mlflow.start_run(run_id=run_id)
run = mlflow.active_run()
print("Active run_id: {}".format(run.info.run_id))
mlflow.log_metric("LB AUC",LB_AUC)
mlflow.end_run()

<ActiveRun: >

Active run_id: 254fd0dba60b46cf84742c32cc677bb4


## big_category_answer log1p 안한것

In [24]:
### 피처 설정
# 사용할 Feature 설정
FEATS = ['uidIdx',
         'assIdx',
         'testIdx',
         'KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
         'solvesec_cumsum',
         'solvecumsum_category',
         'big_category_user_acc',
         'big_category_user_std',
         'big_category_cumconut',
         'big_category_answer'
        ]

cat_feats = ['uidIdx','assIdx','testIdx','KnowledgeTag','big_category','mid_category',
             'problem_num','dayname','month','time_category','solvecumsum_category']
cat_feats_idx = [i for i,e in enumerate(FEATS) if e in cat_feats]

### 학습 및 예측

date = datetime.now().strftime('%m/%d %a')
title=f"🌈({date})[LGBM big_category_answer log1p 안한것] 피처: {len(FEATS)}개"
using_feats=", ".join(FEATS)
desc=f"사용된 피처({len(FEATS)})\n{using_feats}"
cat_feats_idx, title, desc

mlflow.lightgbm.autolog()
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

with mlflow.start_run(run_name=title, description=desc) as run:
    model8 = lgb.train(
        params, 
        lgb_x_train,
        valid_sets=[lgb_x_valid],
        verbose_eval=100,
        num_boost_round=3200,
        categorical_feature=cat_feats_idx,
        early_stopping_rounds=200,
    )

    preds = model8.predict(x_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)
    print(f'VALID AUC : {auc} ACC : {acc}\n')
    mlflow.log_metric("VAL AUC",auc)
    mlflow.log_metric("VAL Set SEED",SEED)
    model8_run_id=run.info.run_id
    post_slack("done")
    print(f"{model8_run_id=}")
    file_name = title2filename(title)
    lgbm_predict(test, model8, FEATS, f'{file_name}.csv')

([0, 1, 2, 3, 6, 7, 8, 9, 11, 21, 23],
 '🌈(12/01 Thu)[LGBM big_category_answer log1p 안한것] 피처: 28개',
 '사용된 피처(28)\nuidIdx, assIdx, testIdx, KnowledgeTag, user_correct_answer, user_total_answer, big_category, mid_category, problem_num, month, day, dayname, hour, user_acc, test_mean, test_sum, test_std, tag_std, tag_mean, tag_sum, solvesec_3600, time_category, solvesec_cumsum, solvecumsum_category, big_category_user_acc, big_category_user_std, big_category_cumconut, big_category_answer')

[LightGBM] [Info] Number of positive: 1208276, number of negative: 637263
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030252 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 20061
[LightGBM] [Info] Number of data points in the train set: 1845539, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654701 -> initscore=0.639767
[LightGBM] [Info] Start training from score 0.639767
Training until validation scores don't improve for 200 rounds
[100]	valid_0's binary_logloss: 0.512626
[200]	valid_0's binary_logloss: 0.503799
[300]	valid_0's binary_logloss: 0.500915
[400]	valid_0's binary_logloss: 0.500644
[500]	valid_0's binary_logloss: 0.500709
[600]	valid_0's binary_logloss: 0.500692
[700]	valid_0's binary_logloss: 0.501092
Early stopping, best iteration is:
[545]	valid_0's binary_logloss: 0.500358
VALID AUC 

In [ ]:
### 제출 mlflow 등록
# 제출시
LB_AUC=0.8112
run_id = model8_run_id
mlflow.start_run(run_id=run_id)
run = mlflow.active_run()
print("Active run_id: {}".format(run.info.run_id))
mlflow.log_metric("LB AUC",LB_AUC)
mlflow.end_run()

## Validation Set Align

In [31]:
# model1
FEATS1 = ['uidIdx',
         'assIdx',
         'testIdx',
         'KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
         'solvesec_cumsum',
         'solvecumsum_category',
         'big_category_acc',
         'big_category_std',
         'big_category_cumconut'
        ]

# model2
FEATS2 = ['uidIdx',
         'assIdx',
         'testIdx',
         'KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
         'solvesec_cumsum',
         'solvecumsum_category',
         'big_category_acc',
         'big_category_std',
         'big_category_cumconut'
        ]

# model5
FEATS5 = ['uidIdx',
         'assIdx',
         'testIdx',
         'KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
         'solvesec_cumsum',
         'solvecumsum_category',
        ]
# model6
FEATS6 = ['uidIdx',
         'assIdx',
         'testIdx',
         'KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
         'solvesec_cumsum',
         'solvecumsum_category',
         'big_category_user_acc',
         'big_category_user_std',
         'big_category_cumconut'
        ]

# model7
FEATS7 = ['uidIdx',
         'assIdx',
         'testIdx',
         'KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
         'solvesec_cumsum',
         'solvecumsum_category',
         'big_category_user_acc',
         'big_category_user_std',
         'big_category_cumconut',
         'big_category_answer_log1p'
        ]

# model71
FEATS71 = ['uidIdx',
         'assIdx',
         'testIdx',
         'KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
         'solvesec_cumsum',
         'solvecumsum_category',
         'big_category_user_acc',
         'big_category_user_std',
         'big_category_cumconut',
         'big_category_answer_log1p'
        ]

In [53]:
def model2auc(model, x_valid, y_valid, FEATS):
    preds = model.predict(x_valid[FEATS])
    auc = roc_auc_score(y_valid, preds)
    return auc

In [34]:
from collections import defaultdict

model_dict = defaultdict(list)

for n in [1,2,5,6,7,71]:
    model_dict[f"model{n}"].append(eval(f'model{n}'))
    model_dict[f"model{n}"].append(eval(f'FEATS{n}'))

In [40]:
model_dict.keys()

dict_keys(['model1', 'model2', 'model5', 'model6', 'model7', 'model71'])

## seed별 학습 및 auc 비교

In [ ]:
model2 # 0.8155 - big_category 정답률, std, cumcount 추가 early stop
model6 # 0.8024 - big_category 유저별 정답률, std, cumcount 추가 early stop
model71 # 0.7948 - big_category_answer_log1p 추가 early stop

In [69]:
def model_train2auc(run_title, x_valid, y_valid, FEATS, seed, experiment_id):
    lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
    lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)
    with mlflow.start_run(run_name=run_title, experiment_id=experiment_id) as run:
        model = lgb.train(
            params, 
            lgb_x_train,
            valid_sets=[lgb_x_valid],
            verbose_eval=-1,
            num_boost_round=3200,
            categorical_feature=cat_feats_idx,
            early_stopping_rounds=200,
        )
        preds = model.predict(x_valid[FEATS])
        auc = roc_auc_score(y_valid, preds)
        mlflow.log_metric("VAL AUC",auc)
        mlflow.log_metric("VAL Set SEED",seed)
    return model, auc

In [ ]:
from mlflow.tracking import MlflowClient
client = MlflowClient()
experiment_name = "Validation Set Align2"
experiment_id = client.create_experiment(experiment_name)

In [65]:
experiment_id

'312762763748397651'

In [70]:
mlflow.lightgbm.autolog()
for s in range(100):
    print(f"----------------------------- SEED {s} -----------------------------")
    train, valid = custom_train_test_split(train_df, ratio=0.7, seed=s) # 훈련데이터 split
    test = df[df.kind=='test'] # 테스트 데이터
    train2 = test[test.answerCode!=-1] # 테스트데이터 마지막 제출 2번쨰꺼까지 훈련데이터로 사용
    train = pd.concat([train,train2]) # 훈련데이터 병합
    x_train = train.drop('answerCode',axis=1)
    y_train = train[['answerCode']]
    x_valid = valid.drop('answerCode',axis=1)
    y_valid = valid[['answerCode']]
    x_train.shape, y_train.shape, x_valid.shape, y_valid.shape
    aucs=[]
    for n,lb in zip([2,6,71],[0.8155,0.8024,0.7948]):
        model_name = f"model{n}"
        run_title = f"model{n} - LB AUC:{lb}"
        model,feats = model_dict[model_name]
        model, auc = model_train2auc(run_title, x_valid, y_valid, feats, s, experiment_id)
        print(f"{model_name} Val Auc: {auc:.3f}")
        model_dict[model_name][0] = model
        aucs.append(auc)
    saucs = sorted(aucs)
    sort_index = [saucs.index(i) for i in aucs]
    print(sort_index)
    if sort_index==[3,4,1,2,0]:
        print(s)
        post_slack("done")
        break

----------------------------- SEED 0 -----------------------------


((1845739, 39), (1845739, 1), (1968, 39), (1968, 1))

[LightGBM] [Info] Number of positive: 1204303, number of negative: 641436
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030596 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19310
[LightGBM] [Info] Number of data points in the train set: 1845739, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.652477 -> initscore=0.629947
[LightGBM] [Info] Start training from score 0.629947
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[760]	valid_0's binary_logloss: 0.525738
model2 Val Auc: 0.812
[LightGBM] [Info] Number of positive: 1204303, number of negative: 641436
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030007 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can

((1845952, 39), (1845952, 1), (2015, 39), (2015, 1))

[LightGBM] [Info] Number of positive: 1208247, number of negative: 637705
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030622 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19249
[LightGBM] [Info] Number of data points in the train set: 1845952, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654539 -> initscore=0.639050
[LightGBM] [Info] Start training from score 0.639050
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[1182]	valid_0's binary_logloss: 0.519259
model2 Val Auc: 0.817
[LightGBM] [Info] Number of positive: 1208247, number of negative: 637705
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029543 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you ca

...

[LightGBM] [Info] Number of positive: 1208434, number of negative: 637374
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028039 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19196
[LightGBM] [Info] Number of data points in the train set: 1845808, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654691 -> initscore=0.639724
[LightGBM] [Info] Start training from score 0.639724
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[827]	valid_0's binary_logloss: 0.525786
model2 Val Auc: 0.813
[LightGBM] [Info] Number of positive: 1208434, number of negative: 637374
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028045 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can

((1845858, 39), (1845858, 1), (2014, 39), (2014, 1))

[LightGBM] [Info] Number of positive: 1207033, number of negative: 638825
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031146 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19287
[LightGBM] [Info] Number of data points in the train set: 1845858, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.653914 -> initscore=0.636290
[LightGBM] [Info] Start training from score 0.636290
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[965]	valid_0's binary_logloss: 0.522364
model2 Val Auc: 0.813
[LightGBM] [Info] Number of positive: 1207033, number of negative: 638825
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028694 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can

((1845722, 39), (1845722, 1), (1980, 39), (1980, 1))

[LightGBM] [Info] Number of positive: 1210271, number of negative: 635451
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027544 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19309
[LightGBM] [Info] Number of data points in the train set: 1845722, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.655717 -> initscore=0.644265
[LightGBM] [Info] Start training from score 0.644265
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[812]	valid_0's binary_logloss: 0.543128
model2 Val Auc: 0.798
[LightGBM] [Info] Number of positive: 1210271, number of negative: 635451
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027392 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can

### seed별 auc 확인

In [52]:
for s in range(100):
    train, valid = custom_train_test_split(train_df, ratio=0.7, seed=s) # 훈련데이터 split
    test = df[df.kind=='test'] # 테스트 데이터
    train2 = test[test.answerCode!=-1] # 테스트데이터 마지막 제출 2번쨰꺼까지 훈련데이터로 사용
    train = pd.concat([train,train2]) # 훈련데이터 병합
    x_train = train.drop('answerCode',axis=1)
    y_train = train[['answerCode']]
    x_valid = valid.drop('answerCode',axis=1)
    y_valid = valid[['answerCode']]
    x_train.shape, y_train.shape, x_valid.shape, y_valid.shape
    aucs=[]
    for n in [1,2,6,7,71]:
        model_name = f"model{n}"
        model,feats = model_dict[model_name]
        auc = model2auc(model, x_valid, y_valid, feats)
        print(f"{model_name} {auc:.3f}",end=' ')
        aucs.append(auc)
    saucs = sorted(aucs)
    sort_index = [saucs.index(i) for i in aucs]
    print(sort_index)
    if sort_index==[3,4,1,2,0]:
        print(s)
        post_slack("done")
        break

((1845739, 39), (1845739, 1), (1968, 39), (1968, 1))

model1 0.888 model2 0.856 model6 0.867 model7 0.870 model71 0.905 [3, 0, 1, 2, 4]


((1845952, 39), (1845952, 1), (2015, 39), (2015, 1))

model1 0.896 model2 0.864 model6 0.871 model7 0.876 model71 0.912 [3, 0, 1, 2, 4]


((1845977, 39), (1845977, 1), (2011, 39), (2011, 1))

...

model1 0.897 model2 0.860 model6 0.868 model7 0.874 model71 0.918 [3, 0, 1, 2, 4]


((1845858, 39), (1845858, 1), (2014, 39), (2014, 1))

model1 0.893 model2 0.863 model6 0.870 model7 0.876 model71 0.910 [3, 0, 1, 2, 4]


((1845722, 39), (1845722, 1), (1980, 39), (1980, 1))

model1 0.887 model2 0.848 model6 0.856 model7 0.860 model71 0.901 [3, 0, 1, 2, 4]


In [48]:
saucs = sorted(aucs)

In [50]:
[saucs.index(i) for i in aucs]==[3,4,1,2,0]

False

In [ ]:
model2 # 0.8155
model6 # 0.8024 
model71 # 0.7948

In [ ]:
model1 # 0.8111 3
model2 # 0.8155 4
model5 # 0.8112
model6 # 0.8024 1
model7 # 0.8062 2
model71 # 0.7948 0

In [ ]:
FEATS1 # 0.8111
FEATS2 # 0.8155
FEATS5 # 0.8112
FEATS6 # 0.8024
FEATS7 # 0.8062
FEATS71 # 0.7948